In [1]:
import pandas as pd
import numpy as np
from builddataset import build_dataset
from raw_processing import label_schemes
import pickle

### Build dataset

In [2]:
# Load dataset
cvs_df = pd.read_csv('../processed_data/AAVSOCVsZTFxmatch_21032023_BTS_manual_labelled.csv')
cvs_df = label_schemes(cvs_df)
print(cvs_df.shape, cvs_df.columns)
cvs_df

# Extract features
features = build_dataset(cvs_df, objcol='oid_ztf', folderpath='../lightcurves_dataset/lasair_2023_03_25', outliercap=False)


(5708, 16) Index(['oid_ztf', 'oid_aavso', 'type_aavso', 'ra', 'dec', 'Eclipsing',
       'CV_Types', 'CV_subtypes', 'CV_subsubtypes', 'eclipse_clear',
       'manual_label', 'Clarity', 'labels_1', 'labels_2', 'labels_3',
       'labels_4'],
      dtype='object')
INFO: Login to gaia TAP server [astroquery.gaia.core]
OK
INFO: Login to gaia data server [astroquery.gaia.core]
OK
500 Error 500:
Can not execute the following SQL: 
DROP TABLE user_dmistry.ast_table
Because: ERROR: table "ast_table" does not exist
500 Error 500:
Can not execute the following SQL: 
DROP TABLE user_dmistry.xmatch_table
Because: ERROR: table "xmatch_table" does not exist
Sending pytable.
Uploaded table 'ast_table'.
Retrieving table 'user_dmistry.ast_table'
Table 'user_dmistry.ast_table' updated.
INFO: Query finished. [astroquery.utils.tap.core]
Table 'ast_table' deleted.
Removed jobs: '['1696103780611O']'.
INFO: Gaia TAP server logout OK [astroquery.gaia.core]
INFO: Gaia data server logout OK [astroquery.gaia.cor

In [3]:
dataset_lcfeats = features[0]
dataset_metafeats = features[1]
dataset_allfeats = features[2]
outlier_thresholds = features[3]

save = True
dataset_date = '21032023'
lc_date = '25032023'
outlier = 'F'

# Save datasets
if save == True:
    dataset_lcfeats.to_csv(f'../processed_data/dataset{dataset_date}_lcfeatures{lc_date}ol{outlier}.csv', index=False)
    dataset_metafeats.to_csv(f'../processed_data/dataset{dataset_date}_gaiafeatures{lc_date}ol{outlier}.csv', index=False)
    dataset_allfeats.to_csv(f'../processed_data/dataset{dataset_date}_allfeatures{lc_date}ol{outlier}.csv', index=False)
    with open(f'../processed_data/outlier_thresholds{dataset_date}_{lc_date}ol{outlier}.pkl', 'wb') as f:
        pickle.dump(outlier_thresholds, f)

In [4]:
dataset_inc_labels = pd.merge(cvs_df[['oid_ztf', 'oid_aavso', 'type_aavso', 'Eclipsing',
                           'CV_Types', 'CV_subtypes', 'CV_subsubtypes', 'eclipse_clear',
                           'manual_label', 'Clarity', 'labels_1', 'labels_2', 'labels_3',
                           'labels_4']],
                           dataset_allfeats, on='oid_ztf', how='left')

if save == True:
    dataset_inc_labels.to_csv(f'../processed_data/dataset{dataset_date}_allfeatures_inc_labels_{lc_date}ol{outlier}.csv', index=False)

In [5]:
data = pd.read_csv(f'../processed_data/dataset{dataset_date}_allfeatures_inc_labels_{lc_date}ol{outlier}.csv')
# identify mixed dtype columns
wierd_cols = []
for col in data.columns:
    weird = (data[[col]].applymap(type) != data[[col]].iloc[0].apply(type)).any(axis=1)
    if len(data[weird]) > 0:
        wierd_cols.append(col)
        print(col)

# convert floats to strings for the weird columns
for col in wierd_cols:
    data[col] = data[col].astype(str)


data.to_csv(f'../processed_data/dataset{dataset_date}_allfeatures_inc_labels_{lc_date}ol{outlier}.csv', index=False)


oid_aavso
CV_Types
CV_subtypes
CV_subsubtypes
manual_label
labels_1
labels_2
labels_3
labels_4
